In [1]:
import pandas as pd
from datetime import datetime, timedelta
import csv

# Function to classify a row based on specified rules
def classify_row(row):
    if (row['Body Temp'] < 35.7 or row['Body Temp'] > 40):
        return 0
    elif (not pd.isna(row['BP Dia']) and not pd.isna(row['BP Sys']) and not pd.isna(row['HRV']) and not pd.isna(row['Stress'])):
        return 1
    elif (not pd.isna(row['SpO2']) and not pd.isna(row['HR'])):
        return 1
    else:
        return 0

# Function to calculate duration based on time difference
def calculate_duration(prev_timestamp, curr_timestamp):
    time_diff = curr_timestamp - prev_timestamp
    if time_diff >= timedelta(minutes=20):
        return timedelta(minutes=10)
    else:
        return time_diff

"""
# Example data stream (replace this with your actual data stream mechanism)
data_stream = [
    {'Time': '2024-02-06 12:00:00', 'SpO2': 95, 'BP Dia': 80, 'BP Sys': 120, 'HRV': 60, 'Stress': 0.1, 'HR': 70, 'Body Temp': 36.5},
    {'Time': '2024-02-06 12:10:00', 'SpO2': None, 'BP Dia': None, 'BP Sys': None, 'HRV': None, 'Stress': None, 'HR': 75, 'Body Temp': 35.5},
    {'Time': '2024-02-06 12:20:00', 'SpO2': 98, 'BP Dia': None, 'BP Sys': None, 'HRV': None, 'Stress': None, 'HR': 80, 'Body Temp': 40.5},
    # Add more data rows here...
]

# Convert data stream to DataFrame
df = pd.DataFrame(data_stream)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Initialize variables for duration calculation
prev_timestamp = None
prev_label = None
label_duration = {'0': timedelta(0), '1': timedelta(0)}

# Process incoming data stream
for index, row in df.iterrows():
    # Classify row
    label = classify_row(row)
    
    # Update label duration
    if prev_timestamp is not None:
        duration = calculate_duration(prev_timestamp, row['Time'])
        label_duration[str(prev_label)] += duration
    
    # Update previous timestamp and label
    prev_timestamp = row['Time']
    prev_label = label

# Calculate duration of label 2 (remaining duration)
total_duration = timedelta(hours=24)
label_2_duration = total_duration - label_duration['0'] - label_duration['1']

# Print label durations
print("Duration of Label 0:", label_duration['0']) # Nyala tidak dipakai
print("Duration of Label 1:", label_duration['1']) # Nyala dipakai
print("Duration of Label 2:", label_2_duration) # Tidak nyala
"""

Duration of Label 0: 0 days 00:10:00
Duration of Label 1: 0 days 00:10:00
Duration of Label 2: 0 days 23:40:00


In [ ]:
# Define the file path
file_path = '2023-12-(04-14)_Label.csv'

# Initialize an empty DataFrame to store daily durations
daily_durations_df = pd.DataFrame(columns=['Date', 'Name', 'Label_0', 'Label_1', 'Label_2'])

# Open the CSV file for reading
with open(file_path, mode='r', newline='') as file:
    reader = csv.DictReader(file)
    
    # Initialize variables to track previous row's data
    prev_name = None
    prev_label = None
    prev_timestamp = None
    
    # Initialize variables to store durations for each Name
    durations = {'0': 0, '1': 0, '2': 0}
    
    # Process each row of the CSV file
    for row in reader:
        # Parse timestamp from string to datetime object
        timestamp = datetime.strptime(row['Time (WIB)'], '%Y-%m-%d %H:%M:%S')
        
        # Check if the Name has changed
        if row['Name'] != prev_name:
            # Add durations for the previous Name to the DataFrame
            if prev_name:
                daily_durations_df = daily_durations_df.append({
                    'Date': prev_timestamp.date(),
                    'Name': prev_name,
                    'Label_0': durations['0'],
                    'Label_1': durations['1'],
                    'Label_2': durations['2']
                }, ignore_index=True)
            
            # Reset durations for the new Name
            durations = {'0': 0, '1': 0, '2': 0}
        
        # Classify the row
        label = classify_row(row)
        
        # Calculate the duration for the current label
        if prev_timestamp:
            duration = (timestamp - prev_timestamp).total_seconds() // 60  # Convert to minutes
            
            # Cap the duration at 10 minutes if the time difference is >= 20 minutes
            if duration >= 20:
                duration = 10
            
            # Update the durations dictionary
            durations[str(prev_label)] += duration
        
        # Update previous row's data
        prev_name = row['Name']
        prev_label = label
        prev_timestamp = timestamp
    
    # Add durations for the last Name to the DataFrame
    if prev_name:
        daily_durations_df = daily_durations_df.append({
            'Date': prev_timestamp.date(),
            'Name': prev_name,
            'Label_0': durations['0'],
            'Label_1': durations['1'],
            'Label_2': durations['2']
        }, ignore_index=True)

# Display the DataFrame
print(daily_durations_df)
